In [1]:
import pandas as pd
import os

In [2]:
#Paths for files

studentspath=os.path.join(".","rawdata","students_complete.csv")
schoolspath=os.path.join(".","rawdata","schools_complete.csv")

In [3]:
#Read students csv and check var names
students_df=pd.read_csv(studentspath)
students_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [4]:
#Read schools csv and check var names
schools_df=pd.read_csv(schoolspath)
schools_df.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [5]:
#Generate variables for the District level summary
schools=schools_df["School ID"].count()
students=students_df["Student ID"].count()
total_budget=schools_df["budget"].sum()
avg_math=students_df["math_score"].sum()/students
avg_reading=students_df["reading_score"].sum()/students

pct_math=students_df.loc[students_df["math_score"]>=70]
pct_math=100*pct_math["math_score"].count()/students

pct_reading=students_df.loc[students_df["reading_score"]>=70]
pct_reading=100*pct_reading["reading_score"].count()/students

pct_overall=(pct_math+pct_reading)/2

#Generate dataframe with District level summary

district_sum={
    "Total Schools":[schools],
    "Total Students":[students],
    "Total Budget":[total_budget],
    "Average Math Score":[avg_math],
    "Average Reading Score":[avg_reading],
    "% Passing Math":[pct_math],
    "% Passing Reading":[pct_reading],
    "% Overall Passing Rate":[pct_overall]
}
district_sum=pd.DataFrame(district_sum)

#Formatting some variables
district_sum["Total Students"]=district_sum["Total Students"].map("{:,}".format)
district_sum["Total Budget"]=district_sum["Total Budget"].map("${:,}".format)
district_sum["Average Math Score"]=district_sum["Average Math Score"].map("{:.2f}".format)
district_sum["Average Reading Score"]=district_sum["Average Reading Score"].map("{:.2f}".format)
district_sum["% Passing Math"]=district_sum["% Passing Math"].map("{:.2f}".format)
district_sum["% Passing Reading"]=district_sum["% Passing Reading"].map("{:.2f}".format)
district_sum["% Overall Passing Rate"]=district_sum["% Overall Passing Rate"].map("{:.2f}".format)

###################################
# Step 1: District Summary
district_sum.head()
###################################

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,80.39


In [6]:
#Starting schools summary with a copy of schools database

budget_perstud=schools_df["budget"]/schools_df["size"]


schools_sum={
    "school_name":pd.Series(schools_df["school_name"]),
    "School Type":pd.Series(schools_df["type"]),
    "Total Students":pd.Series(schools_df["size"]),
    "Total School Budget":pd.Series(schools_df["budget"]),
    "Per Student Budget":budget_perstud
}



schools_sum=pd.DataFrame(schools_sum)

#schools_sum.set_index("School Name")
schools_sum.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget
0,Huang High School,District,2917,1910635,655.0
1,Figueroa High School,District,2949,1884411,639.0
2,Shelton High School,Charter,1761,1056600,600.0
3,Hernandez High School,District,4635,3022020,652.0
4,Griffin High School,Charter,1468,917500,625.0


In [7]:
#Creating series of means of math and reading scores by school name
math_avg=students_df.groupby("school_name")["math_score"].mean()
reading_avg=students_df.groupby("school_name")["reading_score"].mean()

#Counting students with grades above 70 for math and reading and storing in series
math_pass=students_df.loc[students_df["math_score"]>=70,["math_score", "school_name"]]
math_pass=math_pass.groupby("school_name").count()
reading_pass=students_df.loc[students_df["reading_score"]>=70,["reading_score", "school_name"]]
reading_pass=reading_pass.groupby("school_name").count()

#Merging series into a single dataframe
merge_avgpass=pd.merge(math_avg,reading_avg,on="school_name")
merge_avgpass=pd.merge(merge_avgpass,math_pass,on="school_name")
merge_avgpass=pd.merge(merge_avgpass,reading_pass,on="school_name")

merge_avgpass=merge_avgpass.rename(columns={
    "school_name":"School Name",
    "math_score_x":"Average Math Score",
    "reading_score_x":"Average Reading Score",
    "math_score_y":"% Passing Math",
    "reading_score_y":"% Passing Reading",
})
#Create overall passing students
merge_avgpass["% Overall Passing Rate"]=merge_avgpass["% Passing Math"]+merge_avgpass["% Passing Reading"]

#Set remaining variables to float
merge_avgpass["% Passing Math"]=merge_avgpass["% Passing Math"].astype('float64')
merge_avgpass["% Passing Reading"]=merge_avgpass["% Passing Reading"].astype('float64')
merge_avgpass["% Overall Passing Rate"]=merge_avgpass["% Overall Passing Rate"].astype('float64')

merge_avgpass.dtypes

Average Math Score        float64
Average Reading Score     float64
% Passing Math            float64
% Passing Reading         float64
% Overall Passing Rate    float64
dtype: object

In [8]:
#Merging schools dataframe with constructed of scores and pass rates
schools_sum=pd.merge(schools_sum,merge_avgpass,on="school_name")

#Calculate passing rates
schools_sum["% Passing Math"]=100*schools_sum["% Passing Math"]/schools_sum["Total Students"]
schools_sum["% Passing Reading"]=100*schools_sum["% Passing Reading"]/schools_sum["Total Students"]
schools_sum["% Overall Passing Rate"]=100*schools_sum["% Overall Passing Rate"]/(2*schools_sum["Total Students"])

#Formatting vars
#schools_sum["Total Students"]=schools_sum["Total Students"].astype('float64')
#schools_sum["Total School Budget"]=schools_sum["Total School Budget"].astype('float64')
schools_sum.head()

,school_name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


In [9]:
#Formatting vars
schools_sum["Total Students"]=schools_sum["Total Students"].map("{:,}".format)
schools_sum["Total School Budget"]=schools_sum["Total School Budget"].map("${:,}".format)
schools_sum["Per Student Budget"]=schools_sum["Per Student Budget"].map("${:,.0f}".format)
schools_sum["Average Math Score"]=schools_sum["Average Math Score"].map("{:.2f}".format)
schools_sum["Average Reading Score"]=schools_sum["Average Reading Score"].map("{:.2f}".format)
schools_sum["% Passing Math"]=schools_sum["% Passing Math"].map("{:.2f}".format)
schools_sum["% Passing Reading"]=schools_sum["% Passing Reading"].map("{:.2f}".format)
schools_sum["% Overall Passing Rate"]=schools_sum["% Overall Passing Rate"].map("{:.2f}".format)

#Sort dataframe by school name
schools_sum=schools_sum.sort_values(by="school_name", ascending=True)
schools_sum=schools_sum.set_index("school_name")

###################################
#Step 2: Schools summary
schools_sum
###################################

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,73.36
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,94.38
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,73.64


In [10]:
#Selecting schools with lowest overall passing rate
#Sorting dataframe from lowest to highest rate
schools_bottom=schools_sum.sort_values(by="% Overall Passing Rate", ascending=True)
#Keeping only the 5 lowest passing rates
schools_bottom=schools_bottom.head(5)


###################################
#Step 3:Bottom performig schools summary
schools_bottom
###################################

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,73.36
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,73.64
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,73.80


In [11]:
#Creating math and reading average score series by grade

#First extract data by grade
data9=students_df.loc[students_df["grade"]=="9th"]
data10=students_df.loc[students_df["grade"]=="10th"]
data11=students_df.loc[students_df["grade"]=="11th"]
data12=students_df.loc[students_df["grade"]=="12th"]

#Then calculate means of scores
math9=data9.groupby("school_name")["math_score"].mean()
reading9=data9.groupby("school_name")["reading_score"].mean()
math10=data10.groupby("school_name")["math_score"].mean()
reading10=data10.groupby("school_name")["reading_score"].mean()
math11=data11.groupby("school_name")["math_score"].mean()
reading11=data11.groupby("school_name")["reading_score"].mean()
math12=data12.groupby("school_name")["math_score"].mean()
reading12=data12.groupby("school_name")["reading_score"].mean()

#Create dataframes
math_bygrade=pd.merge(math9,math10,on="school_name")
math_bygrade=math_bygrade.rename(columns={
    "math_score_x":"Avg Math Score 9th Grade",
    "math_score_y":"Avg Math Score 10th Grade",
})
math_bygrade=pd.merge(math_bygrade,math11,on="school_name")
math_bygrade=pd.merge(math_bygrade,math12,on="school_name")
math_bygrade=math_bygrade.rename(columns={
    "math_score_x":"Avg Math Score 11th Grade",
    "math_score_y":"Avg Math Score 12th Grade",
})

#Create dataframes
reading_bygrade=pd.merge(reading9,reading10,on="school_name")
reading_bygrade=reading_bygrade.rename(columns={
    "reading_score_x":"Avg Reading Score 9th Grade",
    "reading_score_y":"Avg Reading Score 10th Grade",
})
reading_bygrade=pd.merge(reading_bygrade,reading11,on="school_name")
reading_bygrade=pd.merge(reading_bygrade,reading12,on="school_name")
reading_bygrade=reading_bygrade.rename(columns={
    "reading_score_x":"Avg Reading Score 11th Grade",
    "reading_score_y":"Avg Reading Score 12th Grade",
})

#Formatting vars
math_bygrade["Avg Math Score 9th Grade"]=math_bygrade["Avg Math Score 9th Grade"].map("{:.2f}".format)
math_bygrade["Avg Math Score 10th Grade"]=math_bygrade["Avg Math Score 10th Grade"].map("{:.2f}".format)
math_bygrade["Avg Math Score 11th Grade"]=math_bygrade["Avg Math Score 11th Grade"].map("{:.2f}".format)
math_bygrade["Avg Math Score 12th Grade"]=math_bygrade["Avg Math Score 12th Grade"].map("{:.2f}".format)

reading_bygrade["Avg Reading Score 9th Grade"]=reading_bygrade["Avg Reading Score 9th Grade"].map("{:.2f}".format)
reading_bygrade["Avg Reading Score 10th Grade"]=reading_bygrade["Avg Reading Score 10th Grade"].map("{:.2f}".format)
reading_bygrade["Avg Reading Score 11th Grade"]=reading_bygrade["Avg Reading Score 11th Grade"].map("{:.2f}".format)
reading_bygrade["Avg Reading Score 12th Grade"]=reading_bygrade["Avg Reading Score 12th Grade"].map("{:.2f}".format)



In [12]:
###################################
#Step 4: Scores by grade - math
math_bygrade
###################################

,Avg Math Score 9th Grade,Avg Math Score 10th Grade,Avg Math Score 11th Grade,Avg Math Score 12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [13]:
###################################
#Step 4: Scores by grade - reading
reading_bygrade
###################################

,Avg Reading Score 9th Grade,Avg Reading Score 10th Grade,Avg Reading Score 11th Grade,Avg Reading Score 12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [14]:
all_df=pd.merge(schools_df,students_df,on="school_name")
all_df["perstud"]=all_df["budget"]/all_df["size"]
all_df.head()

upper=max(all_df["perstud"])
lower=min(all_df["perstud"])
range=round((upper-lower)/4)
bins=[lower,lower+range,lower+2*range,lower+3*range,upper]
bins
titles=["$578-$596","$597-$615","$616-$634","$635-$655"]

all_df["Spending Ranges (Per Student)"] = pd.cut(all_df["perstud"], bins, labels=titles)

#Creating series of means of math and reading scores by school name
math_avg=all_df.groupby("Spending Ranges (Per Student)")["math_score"].mean()
reading_avg=all_df.groupby("Spending Ranges (Per Student)")["reading_score"].mean()

#Counting students with grades above 70 for math and reading and storing in series

denom = all_df.groupby("Spending Ranges (Per Student)")["math_score"].count()

math_pass=all_df.loc[all_df["math_score"]>=70,["math_score", "Spending Ranges (Per Student)"]]
math_pass=math_pass.groupby("Spending Ranges (Per Student)").count()
reading_pass=all_df.loc[all_df["reading_score"]>=70,["reading_score", "Spending Ranges (Per Student)"]]
reading_pass=reading_pass.groupby("Spending Ranges (Per Student)").count()

#Merging series into a single dataframe
merge2_avgpass=pd.merge(math_avg,reading_avg,on="Spending Ranges (Per Student)")
merge2_avgpass=pd.merge(merge2_avgpass,math_pass,on="Spending Ranges (Per Student)")
merge2_avgpass=pd.merge(merge2_avgpass,reading_pass,on="Spending Ranges (Per Student)")
merge2_avgpass=merge2_avgpass.rename(columns={
    "math_score_x":"Average Math Score",
    "reading_score_x":"Average Reading Score",
    "math_score_y":"% Passing Math",
    "reading_score_y":"% Passing Reading",
})
#Create overall passing students
merge2_avgpass["% Overall Passing Rate"]=merge2_avgpass["% Passing Math"]+merge2_avgpass["% Passing Reading"]

#Set remaining variables to float
merge2_avgpass["% Passing Math"]=merge2_avgpass["% Passing Math"].astype('float64')
merge2_avgpass["% Passing Reading"]=merge2_avgpass["% Passing Reading"].astype('float64')
merge2_avgpass["% Overall Passing Rate"]=merge2_avgpass["% Overall Passing Rate"].astype('float64')

#Creating % variables
merge2_avgpass["% Passing Math"]=100*merge2_avgpass["% Passing Math"]/denom
merge2_avgpass["% Passing Reading"]=100*merge2_avgpass["% Passing Reading"]/denom
merge2_avgpass["% Overall Passing Rate"]=100*merge2_avgpass["% Overall Passing Rate"]/(2*denom)

#Formatting vars
merge2_avgpass["Average Math Score"]=merge2_avgpass["Average Math Score"].map("{:.2f}".format)
merge2_avgpass["Average Reading Score"]=merge2_avgpass["Average Reading Score"].map("{:.2f}".format)
merge2_avgpass["% Passing Math"]=merge2_avgpass["% Passing Math"].map("{:.2f}".format)
merge2_avgpass["% Passing Reading"]=merge2_avgpass["% Passing Reading"].map("{:.2f}".format)
merge2_avgpass["% Overall Passing Rate"]=merge2_avgpass["% Overall Passing Rate"].map("{:.2f}".format)

###################################
#Step 5:  Scores by School Spending
merge2_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
$578-$596,83.41,83.95,93.61,96.77,95.19
$597-$615,83.53,83.84,94.12,95.89,95.01
$616-$634,78.48,81.67,72.77,85.40,79.08
$635-$655,77.42,81.15,68.34,81.82,75.08


In [15]:
all_df=pd.merge(schools_df,students_df,on="school_name")
all_df["perstud"]=all_df["budget"]/all_df["size"]
all_df.head()

bins=[0,1000,2000,5000]
bins
titles=["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

all_df["School Size"] = pd.cut(all_df["size"], bins, labels=titles)

#Creating series of means of math and reading scores by school name
math_avg=all_df.groupby("School Size")["math_score"].mean()
reading_avg=all_df.groupby("School Size")["reading_score"].mean()

#Counting students with grades above 70 for math and reading and storing in series

denom = all_df.groupby("School Size")["math_score"].count()

math_pass=all_df.loc[all_df["math_score"]>=70,["math_score", "School Size"]]
math_pass=math_pass.groupby("School Size").count()
reading_pass=all_df.loc[all_df["reading_score"]>=70,["reading_score", "School Size"]]
reading_pass=reading_pass.groupby("School Size").count()

#Merging series into a single dataframe
merge3_avgpass=pd.merge(math_avg,reading_avg,on="School Size")
merge3_avgpass=pd.merge(merge3_avgpass,math_pass,on="School Size")
merge3_avgpass=pd.merge(merge3_avgpass,reading_pass,on="School Size")
merge3_avgpass=merge3_avgpass.rename(columns={
    "math_score_x":"Average Math Score",
    "reading_score_x":"Average Reading Score",
    "math_score_y":"% Passing Math",
    "reading_score_y":"% Passing Reading",
})
#Create overall passing students
merge3_avgpass["% Overall Passing Rate"]=merge3_avgpass["% Passing Math"]+merge3_avgpass["% Passing Reading"]

#Set remaining variables to float
merge3_avgpass["% Passing Math"]=merge3_avgpass["% Passing Math"].astype('float64')
merge3_avgpass["% Passing Reading"]=merge3_avgpass["% Passing Reading"].astype('float64')
merge3_avgpass["% Overall Passing Rate"]=merge3_avgpass["% Overall Passing Rate"].astype('float64')

#Creating % variables
merge3_avgpass["% Passing Math"]=100*merge3_avgpass["% Passing Math"]/denom
merge3_avgpass["% Passing Reading"]=100*merge3_avgpass["% Passing Reading"]/denom
merge3_avgpass["% Overall Passing Rate"]=100*merge3_avgpass["% Overall Passing Rate"]/(2*denom)

#Formatting vars
merge3_avgpass["Average Math Score"]=merge3_avgpass["Average Math Score"].map("{:.2f}".format)
merge3_avgpass["Average Reading Score"]=merge3_avgpass["Average Reading Score"].map("{:.2f}".format)
merge3_avgpass["% Passing Math"]=merge3_avgpass["% Passing Math"].map("{:.2f}".format)
merge3_avgpass["% Passing Reading"]=merge3_avgpass["% Passing Reading"].map("{:.2f}".format)
merge3_avgpass["% Overall Passing Rate"]=merge3_avgpass["% Overall Passing Rate"].map("{:.2f}".format)

###################################
#Step 5:  Scores by School Size
merge3_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95,96.04,95.00
Medium (1000-2000),83.37,83.87,93.62,96.77,95.19
Large (2000-5000),77.48,81.20,68.65,82.13,75.39


In [16]:
all_df=pd.merge(schools_df,students_df,on="school_name")
all_df=all_df.rename(columns={"type":"School Type"})

#Creating series of means of math and reading scores by school name
math_avg=all_df.groupby("School Type")["math_score"].mean()
reading_avg=all_df.groupby("School Type")["reading_score"].mean()

#Counting students with grades above 70 for math and reading and storing in series
denom = all_df.groupby("School Type")["math_score"].count()


#Counting students with grades above 70 for math and reading and storing in series
math_pass=all_df.loc[all_df["math_score"]>=70,["math_score", "School Type"]]
math_pass=math_pass.groupby("School Type").count()
reading_pass=all_df.loc[all_df["reading_score"]>=70,["reading_score", "School Type"]]
reading_pass=reading_pass.groupby("School Type").count()

tempie=pd.merge(denom,math_pass,on="School Type")
tempie=pd.merge(tempie,reading_pass,on="School Type")

#Merging series into a single dataframe
merge4_avgpass=pd.merge(math_avg,reading_avg,on="School Type")
merge4_avgpass=pd.merge(merge4_avgpass,math_pass,on="School Type")
merge4_avgpass=pd.merge(merge4_avgpass,reading_pass,on="School Type")

merge4_avgpass=merge4_avgpass.rename(columns={
    "math_score_x":"Average Math Score",
    "reading_score_x":"Average Reading Score",
    "math_score_y":"% Passing Math",
    "reading_score_y":"% Passing Reading",
})

#Create overall passing students
merge4_avgpass["% Overall Passing Rate"]=merge4_avgpass["% Passing Math"]+merge4_avgpass["% Passing Reading"]


#Set remaining variables to float
merge4_avgpass["% Passing Math"]=merge4_avgpass["% Passing Math"].astype('float64')
merge4_avgpass["% Passing Reading"]=merge4_avgpass["% Passing Reading"].astype('float64')
merge4_avgpass["% Overall Passing Rate"]=merge4_avgpass["% Overall Passing Rate"].astype('float64')

#Calculating percentage variables
merge4_avgpass["% Passing Math"]=100*merge4_avgpass["% Passing Math"]/denom
merge4_avgpass["% Passing Reading"]=100*merge4_avgpass["% Passing Reading"]/denom
merge4_avgpass["% Overall Passing Rate"]=100*merge4_avgpass["% Overall Passing Rate"]/(2*denom)

#Formatting vars
merge4_avgpass["Average Math Score"]=merge4_avgpass["Average Math Score"].map("{:.2f}".format)
merge4_avgpass["Average Reading Score"]=merge4_avgpass["Average Reading Score"].map("{:.2f}".format)
merge4_avgpass["% Passing Math"]=merge4_avgpass["% Passing Math"].map("{:.2f}".format)
merge4_avgpass["% Passing Reading"]=merge4_avgpass["% Passing Reading"].map("{:.2f}".format)
merge4_avgpass["% Overall Passing Rate"]=merge4_avgpass["% Overall Passing Rate"].map("{:.2f}".format)

###################################
#Step 5:  Scores by School Type
merge4_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70,96.65,95.17
District,76.99,80.96,66.52,80.91,73.71


##  -------All Results-------

In [17]:
##############Results again##################

In [18]:
###################################
# Step 1: District Summary
district_sum.head()
###################################

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,80.39


In [19]:
###################################
#Step 2: Schools summary
schools_sum
###################################

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628,77.05,81.03,66.68,81.93,74.31
Cabrera High School,Charter,"1,858","$1,081,356",$582,83.06,83.98,94.13,97.04,95.59
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,73.36
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,73.80
Griffin High School,Charter,"1,468","$917,500",$625,83.35,83.82,93.39,97.14,95.27
Hernandez High School,District,"4,635","$3,022,020",$652,77.29,80.93,66.75,80.86,73.81
Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51,96.25,94.38
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,73.64


In [20]:
###################################
#Step 3:Bottom performig schools summary
schools_bottom
###################################

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363",$637,76.84,80.74,66.37,80.22,73.29
Figueroa High School,District,"2,949","$1,884,411",$639,76.71,81.16,65.99,80.74,73.36
Huang High School,District,"2,917","$1,910,635",$655,76.63,81.18,65.68,81.32,73.50
Johnson High School,District,"4,761","$3,094,650",$650,77.07,80.97,66.06,81.22,73.64
Ford High School,District,"2,739","$1,763,916",$644,77.10,80.75,68.31,79.30,73.80


In [21]:
###################################
#Step 4: Scores by grade - math
math_bygrade
###################################

,Avg Math Score 9th Grade,Avg Math Score 10th Grade,Avg Math Score 11th Grade,Avg Math Score 12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


In [22]:
###################################
#Step 4: Scores by grade - reading
reading_bygrade
###################################

,Avg Reading Score 9th Grade,Avg Reading Score 10th Grade,Avg Reading Score 11th Grade,Avg Reading Score 12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


In [23]:
###################################
#Step 5:  Scores by School Spending
merge2_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
$578-$596,83.41,83.95,93.61,96.77,95.19
$597-$615,83.53,83.84,94.12,95.89,95.01
$616-$634,78.48,81.67,72.77,85.40,79.08
$635-$655,77.42,81.15,68.34,81.82,75.08


In [24]:
###################################
#Step 5:  Scores by School Size
merge3_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.83,83.97,93.95,96.04,95.00
Medium (1000-2000),83.37,83.87,93.62,96.77,95.19
Large (2000-5000),77.48,81.20,68.65,82.13,75.39


In [25]:
###################################
#Step 5:  Scores by School Type
merge4_avgpass
###################################

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.41,83.90,93.70,96.65,95.17
District,76.99,80.96,66.52,80.91,73.71


### Comments regarding the students data

* There is a negative correlation between spending per student and average math attainment. The higher the spending per student, the lower the math scores and % pass rates. 
* There is a negative correlation between school size and average math attainment. Schools belonging to the "large" category,have significant lower math score averages and % pass rates.
* Charter schools do better on average on math scores with respecto to District schools.

- - -